Напишите функцию для подсчета количества параметров в сверточном слое. Ваша функция должна принимать на вход количество входных каналов, количество выходных каналов, размер ядра и стоит ли делать обучаемый сдвиг. Сделайте для нее такую сигнатуру: def count_parameters_conv(in_channels: int, out_channels: int, kernel_size: int, bias: bool):

In [ ]:
def count_parameters_conv(in_channels: int, out_channels: int, kernel_size: int, bias: bool):
     # Подсчет параметров для весов и сдвига по формуле: (C1 * n^2 + 1) * C2
    num_parameters = (in_channels * kernel_size**2) * out_channels
    return num_parameters

Обучите полносвязную нейронную сеть для классификации на датасете MNIST. Добейтесь качества в 98% на тестовой выборке.

Можете использовать уже написанные ранее функции train и evaluate.

Чтобы мы могли проверить вашу модель, напишите функцию, которая создает вашу модель и возвращает объект, назовите функцию create_mlp_model, без аргументов. Вам понадобится сохранить веса вашей модели и сдать их в тестировщик, для этого воспользуйтесь методами torch.save и state_dict.

Чекер ожидает от вас два файла. Первый файл с функцией create_mlp_model в формате .py. Второй файл с весами в формате .pt.



In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torchvision.transforms as T
from torch.optim import Optimizer, Adam



def train(model: nn.Module, data_loader: DataLoader, optimizer: Optimizer, loss_fn) -> tuple[float, float]:
    model.train()

    total_loss = 0
    total = 0
    correct = 0

    for x, y in tqdm(data_loader, desc='Train'):
        optimizer.zero_grad()

        output = model(x)

        loss = loss_fn(output, y)

        loss.backward()

        total_loss += loss.item()

        optimizer.step()

        _, y_pred = torch.max(output, 1)
        total += y.size(0)
        correct += (y_pred == y).sum().item()

    return total_loss / len(data_loader), correct / total

In [2]:
def evaluate(model: nn.Module, data_loader: DataLoader, loss_fn) -> tuple[float, float]:
    model.eval()

    total_loss = 0
    total = 0
    correct = 0

    for x, y in tqdm(data_loader, desc='Evaluate'):
        output = model(x)

        loss = loss_fn(output, y)

        total_loss += loss.item()

        _, y_pred = torch.max(output, 1)
        total += y.size(0)
        correct += (y_pred == y).sum().item()

    return total_loss / len(data_loader), correct / total

In [8]:
from torch import nn

def create_mlp_model():
    model = nn.Sequential(
        nn.Flatten(),  # Преобразуем изображение 28x28 в вектор длины 784
        nn.Linear(28 * 28, 512),  # Первый скрытый слой
        nn.ReLU(),  # Функция активации ReLU
        nn.Linear(512, 256),  # Второй скрытый слой
        nn.ReLU(),  # Функция активации ReLU
        nn.Linear(256, 10)  # Выходной слой для 10 классов
    )
    return model

In [5]:
from tqdm import tqdm

# Загружаем тренировочные и тестовые данные
train_dataset = datasets.MNIST(root="../datasets/mnist",
                               train=True,
                               download=True,
                               transform=T.ToTensor())

test_dataset = datasets.MNIST(root="../datasets/mnist",
                              train=False,
                              download=True,
                              transform=T.ToTensor())


train_loader = DataLoader(train_dataset,
                          batch_size=64,
                          shuffle=True)

test_loader = DataLoader(test_dataset,
                          batch_size=64,
                          shuffle=False)


model = create_mlp_model()

# Определяем функцию потерь и оптимизатор
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


#Train the model
for epoch in range(10):
    train_loss, train_accuracy = train(model, train_loader, optimizer, loss_fn)
    print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Train Accuracy = {train_accuracy}%")

# Оценка модели на тестовой выборке
test_loss, test_accuracy = evaluate(model, test_loader, loss_fn)
print(f"Test Loss = {test_loss}, Test Accuracy = {test_accuracy}")

NameError: name 'test_loss' is not defined

In [14]:
# Сохраняем веса модели в директории /kaggle/working/
torch.save(model.state_dict(), '/kaggle/working/mnist_mlp_model.pth')

Обучите сверточную нейронную сеть для классификации на датасете MNIST. Добейтесь качества в 99.3% на тестовой выборке.

Можете использовать уже написанные ранее функции train и evaluate.

Чтобы мы могли проверить вашу модель, напишите функцию, которая создает вашу модель и возвращает объект, назовите функцию create_conv_model, без аргументов. Вам понадобится сохранить веса вашей модели и сдать их в тестировщик, для этого воспользуйтесь методами torch.save и state_dict.

In [17]:
from torch import nn

def create_conv_model():
    model = nn.Sequential(
        # Первый сверточный слой
        nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),  # Padding чтобы сохранить размер
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),

        # Второй сверточный слой
        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),  # Padding чтобы сохранить размер
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),

        # Третий сверточный слой
        nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),  # Padding чтобы сохранить размер
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),

        # Четвертый сверточный слой
        nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),  # Padding чтобы сохранить размер
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        
        # Полносвязные слои
        nn.Flatten(),
        nn.Linear(256 * 1 * 1, 512),  # Уменьшили размер после последнего слоя свертки
        nn.ReLU(),
        nn.Linear(512, 10)  # 10 классов для MNIST
    )
    return model


In [18]:
from tqdm import tqdm

# Загружаем тренировочные и тестовые данные
train_dataset = datasets.MNIST(root="../datasets/mnist",
                               train=True,
                               download=True,
                               transform=T.ToTensor())

test_dataset = datasets.MNIST(root="../datasets/mnist",
                              train=False,
                              download=True,
                              transform=T.ToTensor())

train_loader = DataLoader(train_dataset,
                         batch_size=64,
                         shuffle=True)

test_loader = DataLoader(test_dataset,
                        batch_size=64,
                        shuffle=False)

model = create_conv_model()

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

for epoch in range(20):
    train_loss, train_accuracy = train(model, train_loader, optimizer, loss_fn)
    print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Train Accuracy = {train_accuracy}")


test_loss, test_accuracy = evaluate(model, test_loader, loss_fn)
print(f"Test Loss = {test_loss}, Test Accuracy = {test_accuracy}")

Train: 100%|██████████| 938/938 [00:48<00:00, 19.19it/s]


Epoch 1: Train Loss = 0.2517, Train Accuracy = 0.9186833333333333


Train: 100%|██████████| 938/938 [00:48<00:00, 19.42it/s]


Epoch 2: Train Loss = 0.0605, Train Accuracy = 0.9816333333333334


Train: 100%|██████████| 938/938 [00:48<00:00, 19.18it/s]


...

Epoch 19: Train Loss = 0.0056, Train Accuracy = 0.9980666666666667


Train: 100%|██████████| 938/938 [00:48<00:00, 19.17it/s]


Epoch 20: Train Loss = 0.0041, Train Accuracy = 0.99865


Evaluate: 100%|██████████| 157/157 [00:03<00:00, 42.09it/s]

Test Loss = 0.030202927676024553, Test Accuracy = 0.9936


In [19]:
torch.save(model.state_dict(), '/kaggle/working/mnist_mlp_model_5.pt')